<a href="https://colab.research.google.com/github/Chan0226/Corona-chart/blob/main/%EC%9D%B8%EA%B3%B5%EC%8B%A0%EA%B2%BD%EB%A7%9D%EC%9D%98_%EC%84%B1%EB%8A%A5%ED%8F%89%EA%B0%80_%EC%9F%81%EC%A0%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 성능평가를 하는 이유
  # 1. 하이퍼 매개변수 최적화
  # 2. 현장 설치 여부를 결정
# 객관적인 성능 평가 방법
  # 1 교차검증
  # 2 제거조사(ablation study)
    # 여러선택사항을 다 적용하고 하나씩 빼서 성능을 측정한후 각각의 기여도를 조사

In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from tensorflow.keras import regularizers

In [ ]:
# 데이터 셋 읽고 신경망에 맞도록 변경
(x_train,y_train),(x_test,y_test) =  cifar10.load_data()
x_train = x_train.astype(np.float32)/255.0
x_test = x_test.astype(np.float32)/255.0
y_train = tf.keras.utils.to_categorical(y_train,10)
y_test = tf.keras.utils.to_categorical(y_test,10)

170508288/170498071 [==============================] - 2s 0us/step


In [ ]:
# 하이퍼 매개변수
batch_size = 128
n_epoch = 10
k = 5

In [ ]:
# 하이퍼 매개변수에 따라서 교차검증 수행하고 정확율 반환하는 함수
# 데이터증가, 드랍아웃 비율, 가중치 감소
def cross_validation(data_gen,dropout_rate,l2_reg):
  acurracy = []
  for train_index, val_index in  KFold(k).split(x_train):
    # 훈련 집합 검증 집합
    xtrain,xval = x_train[train_index], x_train[val_index]
    ytrain,yval = y_train[train_index], y_train[val_index]

    # 신경망 모델 설계  C C P D  C C P D FC D FC
    cnn = Sequential()
    cnn.add(Conv2D(32,(3,3),activation='relu', input_shape=(32,32,3) ))
    cnn.add(Conv2D(32,(3,3),activation='relu' ))
    cnn.add(MaxPooling2D(pool_size=(2,2)))
    cnn.add(Dropout(dropout_rate[0]))

    cnn.add(Conv2D(64,(3,3),activation='relu'))
    cnn.add(Conv2D(64,(3,3),activation='relu' ))
    cnn.add(MaxPooling2D(pool_size=(2,2)))
    cnn.add(Dropout(dropout_rate[1]))

    cnn.add(Flatten())
    cnn.add(Dense(512,activation='relu'))
    cnn.add(Dropout(dropout_rate[2]))
    cnn.add(Dense(10,activation='softmax',
                  kernel_regularizer=regularizers.l2(l2_reg)
                  ))  # 이진 분류 sigmoid

    # 학습
    # 신경망 모델 학습
    cnn.compile(loss = 'categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])        
    if data_gen:
      generator = ImageDataGenerator(rotation_range=3.0,width_shift_range=0.1,
                                     height_shift_range = 0.1,horizontal_flip=True)
      cnn.fit.generator(generator.flow(x_train,y_train,batch_size=batch_size,epochs=n_epoch),
                            validation_data = (x_test,y_test), verbose=2)
    else:
      cnn.fit(x_train,y_train,batch_size=batch_size,epochs=n_epoch,
                            validation_data = (x_test,y_test), verbose=2)
    acurracy.append(cnn.evaluate(xval,yval,verbose=0)[1] )
  return acurracy

In [ ]:
# 하이퍼 매개변수를 달리 하면서 신경망 모델을 평가
acc_000 = cross_validation(False,[0.0,0.0,0.0],0.0)         #0 0 0
acc_001 = cross_validation(False,[0.0,0.0,0.0],0.01)        #0 0 1
acc_010 = cross_validation(False,[0.25,0.25,0.5],0.00)      #0 1 0
acc_011 = cross_validation(False,[0.25,0.25,0.5],0.01)       #0 1 1
acc_100 = cross_validation(True,[0.0,0.0,0.0],0.00)       #1 0 0
acc_101 = cross_validation(True,[0.0,0.0,0.0],0.01)       #1 0 1
acc_110 = cross_validation(True,[0.25,0.25,0.5],0.00)       #1 1 0
acc_111 = cross_validation(True,[0.25,0.25,0.5],0.01)       #1 1 1

print("000",acc_000,' : ', np.array(acc_000).mean())
print("000",acc_001,' : ', np.array(acc_001).mean())
print("000",acc_010,' : ', np.array(acc_010).mean())
print("000",acc_011,' : ', np.array(acc_011).mean())
print("000",acc_100,' : ', np.array(acc_100).mean())
print("000",acc_101,' : ', np.array(acc_101).mean())
print("000",acc_110,' : ', np.array(acc_110).mean())
print("000",acc_111,' : ', np.array(acc_111).mean())
 
# boxplot으로 정확율 표시
plt.grid()
plt.plot([acc_000,acc_000,acc_000,acc_000,acc_000,acc_000,acc_000,acc_000],
         labels=['acc_000','acc_000','acc_000','acc_000','acc_000','acc_000','acc_000','acc_000'])






Epoch 1/10
391/391 - 37s - loss: 1.5002 - accuracy: 0.4571 - val_loss: 1.2099 - val_accuracy: 0.5658 - 37s/epoch - 95ms/step
Epoch 2/10
391/391 - 7s - loss: 1.0746 - accuracy: 0.6218 - val_loss: 1.0087 - val_accuracy: 0.6431 - 7s/epoch - 17ms/step
Epoch 3/10
391/391 - 7s - loss: 0.8759 - accuracy: 0.6958 - val_loss: 0.8655 - val_accuracy: 0.6989 - 7s/epoch - 17ms/step
Epoch 4/10
391/391 - 7s - loss: 0.7423 - accuracy: 0.7415 - val_loss: 0.7786 - val_accuracy: 0.7308 - 7s/epoch - 17ms/step
Epoch 5/10
391/391 - 7s - loss: 0.6300 - accuracy: 0.7815 - val_loss: 0.7514 - val_accuracy: 0.7439 - 7s/epoch - 17ms/step
Epoch 6/10
391/391 - 7s - loss: 0.5348 - accuracy: 0.8136 - val_loss: 0.7481 - val_accuracy: 0.7471 - 7s/epoch - 17ms/step
Epoch 7/10
391/391 - 7s - loss: 0.4497 - accuracy: 0.8410 - val_loss: 0.7480 - val_accuracy: 0.7552 - 7s/epoch - 17ms/step
Epoch 8/10
391/391 - 7s - loss: 0.3644 - accuracy: 0.8715 - val_loss: 0.7840 - val_accuracy: 0.7553 - 7s/epoch - 17ms/step
Epoch 9/10
391